In [1]:
import numpy as np # linear algebra
import pandas as pd
import requests,holidays
import xml.etree.ElementTree as et 
from datetime import date as d

def is_ecb_holiday_or_weekend():
    return  d.today() in holidays.ECB() or d.today().weekday() >=5 

#Read ecb xml
user_agent_url = 'https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/pln.xml'
xml_data = requests.get(user_agent_url).content
root = et.fromstring(xml_data)
for Series in root[1].findall('{http://www.ecb.europa.eu/vocabulary/stats/exr/1}Series'):
    df=pd.DataFrame(Series[len(Series)-15].attrib,index=[0])
    for Obs in Series[len(Series)-14:]:
        df=df.append(Obs.attrib, ignore_index=True )

# Transform columns
df.rename(columns={"TIME_PERIOD": "date", 'OBS_STATUS':'currency_base_id',"OBS_CONF": "currency_id",'OBS_VALUE':'exchange_rate'},inplace=True)
df=df[['date','currency_base_id','currency_id','exchange_rate']]
df.replace({'A': 1, 'F': 2},inplace=True)

# Adjust and fill the holidays with the previous row
df['date'] = df['date'].astype('datetime64[ns]')
max_date =  d.today() if is_ecb_holiday_or_weekend() else max(df.date)
idx = pd.date_range(min(df.date), max_date)

df=df.set_index('date').reindex(idx).rename_axis('date').reset_index()
df.fillna(method='pad',inplace = True)
df['currency_base_id'] = df['currency_base_id'].astype('int64')
df['currency_id'] = df['currency_id'].astype('int64')
df['exchange_rate'] = df['exchange_rate'].astype('float')
df=df.iloc[-15:,:] #the last 15 days' rates
df

,date,currency_base_id,currency_id,exchange_rate
6,2020-01-28,1,2,4.2731
7,2020-01-29,1,2,4.2837
8,2020-01-30,1,2,4.2873
9,2020-01-31,1,2,4.3009
10,2020-02-01,1,2,4.3009
11,2020-02-02,1,2,4.3009
12,2020-02-03,1,2,4.2968
13,2020-02-04,1,2,4.2753
14,2020-02-05,1,2,4.2491
15,2020-02-06,1,2,4.2451
